In [ ]:
#default_exp utils

# utils
> Contains various utility functions

In [ ]:
#export
import torch
import numpy as np
from torch import Tensor
from fastcore.utils import is_listy, is_iter

In [ ]:
from fastcore.test import test_eq

In [ ]:
#export
def noop(x=None, *args,**kwargs):
    "Do nothing"
    return x

In [ ]:
#export
def apply(func, x, *args, **kwargs):
    "Apply `func` recursively to `x`, passing on args"
    if is_listy(x): return type(x)([apply(func, o, *args, **kwargs) for o in x])
    if isinstance(x,dict):  return {k: apply(func, v, *args, **kwargs) for k,v in x.items()}
    res = func(x, *args, **kwargs)
    return res if x is None else retain_type(res, x)

In [ ]:
#export
def to_device(b, device='cpu'):
    "Recursively put `b` on `device`."
    def _inner(o): return o.to(device, non_blocking=True) if isinstance(o,Tensor) else o.to_device(device) if hasattr(o, "to_device") else o
    return apply(_inner, b)

In [ ]:
def tensor(x, *rest, **kwargs):
    "Like `torch.as_tensor`, but handle lists too, and can pass multiple vector elements directly."
    if len(rest): x = (x,)+rest
    res = (x if isinstance(x, Tensor)
           else torch.tensor(x, **kwargs) if isinstance(x, (tuple,list))
           else _array2tensor(x) if isinstance(x, np.ndarray)
           else as_tensor(x, **kwargs) if hasattr(x, '__array__') or is_iter(x)
           else _array2tensor(array(x), **kwargs))
    if res.dtype is torch.float64: return res.float()
    return res

In [ ]:
#export
def _array2tensor(x):
    if x.dtype==np.uint16: x.astype(np.float32)
    return torch.from_numpy(x)

In [ ]:
test_eq(_array2tensor(np.array([3,2,1])), tensor([3,2,1]))